# CS572 Project
* We selected the paper: {}
* Based on the data and model of given paper, we are planning to make an extension of LSTM that predicts {}.

## Preparing

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import time
from matplotlib import pyplot as plt

In [2]:
input_size = 8
hidden_size = 128
num_layers = 3
num_epochs = 2000
batch_size = 32
learning_rate = 0.001
weight_decay=0.00001

Check if your computer is prepared to run pytorch model with CUDA.

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.x7 = torch.ones(1).to(device)
        
        self.lstm_cell_1 = nn.LSTMCell(input_size, hidden_size)
        self.lstm_cell_2 = nn.LSTMCell(hidden_size, hidden_size)
        self.lstm_cell_3 = nn.LSTMCell(hidden_size, hidden_size)
        self.BT1_power = nn.Parameter(torch.zeros(hidden_size))
        self.BT2_power = nn.Parameter(torch.zeros(hidden_size))
        self.BT3_power = nn.Parameter(torch.zeros(hidden_size))
        # x -> batch_size, seq, input_size
        self.fc = nn.Linear(hidden_size, 1)
    
    def makehc_zero(self, batch_size):
        self.hn_1 = torch.zeros(batch_size, self.hidden_size).to(device)
        self.cn_1 = torch.zeros(batch_size, self.hidden_size).to(device)

        self.hn_2 = torch.zeros(batch_size, self.hidden_size).to(device)
        self.cn_2 = torch.zeros(batch_size, self.hidden_size).to(device)

        self.hn_3 = torch.zeros(batch_size, self.hidden_size).to(device)
        self.cn_3 = torch.zeros(batch_size, self.hidden_size).to(device)
    
    def forward(self, x):
        if self.training == True:
            # Set initial hidden and cell states 
            self.makehc_zero(x.size(0))
        
#         out, _ = self.rnn(x, h0)
        
        # Forward propagate LSTM
        out_seq = []
        for i in range(x.size(1)): #find seq size
            if self.training == True:
                if i == 0:
                    BT = x[:,i,6]
                else:
                    BT = torch.sub(x[:,i,6], x[:,i-1,6])
                BT = torch.mul(BT, (i+1)/40)
            else:
                BT = torch.sub(x[:,0,6], self.x7)
                self.x7 = x[:,0,6]
                BT = torch.mul(BT, 1)
#             print("before pow: ", BT)
            BT = torch.mul(BT.view(-1,1), torch.ones(1, self.hidden_size).to(device))
#             print(BT)
            BT_1 = torch.pow(BT, self.BT1_power)
            BT_2 = torch.pow(BT, self.BT2_power)
            BT_3 = torch.pow(BT, self.BT3_power)
    
            self.hn_1, self.cn_1 = self.lstm_cell_1(x[:,i,:], (self.hn_1, self.cn_1))  # out: tensor of shape (batch_size, seq_length, hidden_size)
            self.hn_1 = torch.add(self.hn_1, BT_1)
            self.cn_1 = torch.add(self.cn_1, BT_1)
            
            self.hn_2, self.cn_2 = self.lstm_cell_2(self.hn_1, (self.hn_2, self.cn_2))
            self.hn_2 = torch.add(self.hn_2, BT_2)
            self.cn_2 = torch.add(self.cn_2, BT_2)
            
            self.hn_3, self.cn_3 = self.lstm_cell_3(self.hn_2, (self.hn_3, self.cn_3))
            self.hn_3 = torch.add(self.hn_3, BT_3)
            self.cn_3 = torch.add(self.cn_3, BT_3)
            
            out_1 = self.fc(self.hn_3)
            print(out_1)
            out_seq += [out_1]
        out = torch.cat(out_seq, dim=1)
        
        # Decode the hidden state of the last time step
        # out : batch_size, seq_length, hidden_size
        # out (N, 9, 128)  
#         out = hn_3[:, -1, :]
        
        
        return out

Data Loading

In [5]:
training_set = []
data_path = 'Data\Training' # add path

for path_idx in os.listdir(data_path):
    cur_path = os.path.join(data_path, path_idx)
    print(cur_path)
    input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
    # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
    inputs = torch.Tensor(input_file)
#     print(inputs[0])
    
    inputs = inputs[:len(inputs)-len(inputs)%40]
#     labels = inputs[:, [5]]
# #     print(labels.shape)
#     labels = labels.view([-1, 40, 1])
    
    inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
    inputs = inputs.view([-1,40,9])
    
#     for test a file
    training_set += inputs

testing_set = []
data_path = 'Data\Testing' # add path
for path_idx in os.listdir(data_path):
    cur_path = os.path.join(data_path, path_idx)
    print(cur_path)
    input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
    # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
    inputs = torch.Tensor(input_file)
#     print(inputs[0])
    
    inputs = inputs[:len(inputs)-len(inputs)%40]
#     labels = inputs[:, [5]]
# #     print(labels.shape)
#     labels = labels.view([-1, 40, 1])
    
    inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
    inputs = inputs.view([-1,40,9])
    
#     for test a file
    testing_set += inputs
   
# print(data_set)
print("load complete")

Data\Training\190722_tension_test_space_1_trial_2.csv
Data\Training\190722_tension_test_space_1_trial_3.csv
Data\Training\190722_tension_test_space_2_trial_1.csv
Data\Training\190722_tension_test_space_2_trial_2.csv
Data\Training\190722_tension_test_space_2_trial_4.csv
Data\Training\190722_tension_test_space_3_trial_2.csv
Data\Training\190722_tension_test_space_3_trial_3.csv
Data\Training\190722_tension_test_space_3_trial_4.csv
Data\Training\190722_tension_test_space_3_trial_5.csv
Data\Training\190722_tension_test_space_3_trial_6.csv
Data\Training\190722_tension_test_space_4_trial_2.csv
Data\Training\190722_tension_test_space_4_trial_3.csv
Data\Training\190722_tension_test_space_5_trial_2.csv
Data\Training\190722_tension_test_space_5_trial_3.csv
Data\Training\test11thick_1.csv
Data\Training\test11thick_3.csv
Data\Training\test22thick_2.csv
Data\Training\test33thick_2.csv
Data\Training\test33thick_3.csv
Data\Training\test55thick_2.csv
Data\Training\test55thick_3.csv
Data\Testing\190722_

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=training_set,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=testing_set,
                                          batch_size=1, 
                                          shuffle=False)
print("loader setting complete")

loader setting complete


In [14]:
# Training and Testing: To train fingertip forces from input
# data, a three-layer BT-LSTM with 128 hidden units is used.
# Sequence time length of BT-LSTM was set to T = 40. The final
# output of the BT-LSTM layer is fed into the final fully-connected
# layer to obtain the one-dimensional output. The mean squared
# error (MSE) function is used to measure the loss between the
# predicted and the ground truth contact force. During training,
# the number of epoch was 2000, and the mini batch size was 32.
# Adam optimizer was used with 0.001 learning rate and 10−5
# weight decay
lstm = LSTM(input_size, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=weight_decay)

Train model

In [ ]:
for i, x in enumerate(train_loader):
#     print (i, x)
#     print(x.shape)
#     inputs, labels = torch.split(x, [8, 1], 2)
#     print(inputs.shape)
#     print(labels.shape)
    break

In [ ]:
for i in test_loader:
    print (i)
    break

In [15]:
# Train the model
lstm.train()
start = time.time()
print("training start")
num_epochs = 200
for epoch in range(num_epochs):
    for i, inputs in enumerate(train_loader):
        inputs, labels = torch.split(inputs, [8, 1], 2)
        inputs = inputs.to(device)
        labels = labels.squeeze().to(device)
        
        # Forward pass
        outputs = lstm(inputs)
        
#         print(labels.shape)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())
        
        
        
#     if (epoch+1) % 10 == 0:
    print ('Epoch [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, loss.item()))
    print('elapsed time :', time.time() - start)
print("training complete")
print('total_training elapsed time :', time.time() - start)

training start
tensor([[0.0350],
        [0.0377],
        [0.0353],
        [0.0377],
        [0.0365],
        [0.0350],
        [0.0363],
        [0.0373],
        [0.0340],
        [0.0335],
        [0.0337],
        [0.0364],
        [0.0338],
        [0.0337],
        [0.0378],
        [0.0369],
        [0.0378],
        [0.0374],
        [0.0329],
        [0.0336],
        [0.0345],
        [0.0343],
        [0.0358],
        [0.0337],
        [0.0360],
        [0.0340],
        [0.0368],
        [0.0339],
        [0.0356],
        [0.0334],
        [0.0361],
        [0.0346]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[0.0026],
        [0.0065],
        [0.0027],
        [0.0065],
        [0.0053],
        [0.0031],
        [0.0041],
        [0.0061],
        [0.0012],
        [0.0017],
        [0.0022],
        [0.0052],
        [0.0021],
        [0.0013],
        [0.0062],
        [0.0044],
        [0.0085],
        [0.0056],
        [0.0010],
        [0.0020],
      

tensor([[-0.2086],
        [-0.2062],
        [-0.2081],
        [-0.2064],
        [-0.2103],
        [-0.2076],
        [-0.2096],
        [-0.2048],
        [-0.2086],
        [-0.2088],
        [-0.2102],
        [-0.2112],
        [-0.2103],
        [-0.2073],
        [-0.2072],
        [-0.2080],
        [-0.2026],
        [-0.2098],
        [-0.2069],
        [-0.2098],
        [-0.2079],
        [-0.2089],
        [-0.2097],
        [-0.2081],
        [-0.2086],
        [-0.2086],
        [-0.2085],
        [-0.2081],
        [-0.2069],
        [-0.2089],
        [-0.2135],
        [-0.2094]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.2087],
        [-0.2063],
        [-0.2081],
        [-0.2066],
        [-0.2104],
        [-0.2077],
        [-0.2096],
        [-0.2049],
        [-0.2086],
        [-0.2089],
        [-0.2103],
        [-0.2113],
        [-0.2104],
        [-0.2074],
        [-0.2074],
        [-0.2081],
        [-0.2030],
        [-0.2099],
       

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan

        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],

        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],

...

        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],

        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],

nan
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        

        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],

KeyboardInterrupt: 

Test model

In [ ]:
lstm.eval()
start = time.time()
with torch.no_grad():
    total_labels = []
    total_outputs = []
    lstm.makehc_zero(1) #test batch size = 1
    for inputs in test_loader:
        inputs, labels = torch.split(inputs, [8, 1], 2)
        
        
        inputs = inputs.to(device)
        labels = labels.squeeze()
        
        for i in range(inputs.size(1)):
            outputs = lstm(inputs[:,i,:].view([inputs.size(0),1,8]))
            total_outputs += outputs.cpu()
        
        total_labels += labels
    rmse = mean_squared_error(total_labels, total_outputs, squared = False)
    mae = mean_absolute_error(total_labels, total_outputs)

    print('Test Accuracy of the model RMSE: {}'.format(rmse))
    print('Test Accuracy of the model MAE: {}'.format(mae)) 
    print('testing elapsed time :', time.time() - start)

Save model

In [ ]:
torch.save(lstm.state_dict(), 'LSTM_cell_200epoch.ckpt')
print("save complete")

Plot model

In [ ]:
plt.plot(total_outputs)
plt.plot(total_labels)
plt.show()

In [ ]:
plt.plot(total_outputs)
plt.plot(total_labels)
plt.savefig('LSTM_cell_compare_all_labels_200epoch.png')